In [11]:
import requests
import pandas as pd

In [12]:
HEADERS = {
    'Host': 'data.nba.net',
    'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10.15; rv:79.0) Gecko/20100101 Firefox/79.0',
    'Accept': 'application/json, text/plain, */*',
    'Accept-Language': 'en-US,en;q=0.5',
    'Accept-Encoding': 'gzip, deflate, br',
    'Connection': 'keep-alive',
    'api-key': 'PF2JvsskTXRs!rvmkU*.8YXg96DLKE3GN'
}

In [13]:
livescore = requests.get('https://fantasy.trashtalk.co/API/v1/livescore',
                                     headers=HEADERS,
                                     proxies=None).json()

In [14]:
injuries = requests.get('https://www.rotowire.com/basketball/tables/injury-report.php?team=ALL&pos=ALL')

In [33]:
injuries_df = pd.DataFrame(injuries.json())

In [31]:
today_games = requests.get('https://cdn.nba.com/static/json/liveData/scoreboard/todaysScoreboard_00.json').json()['scoreboard']['games']

In [32]:
teams_abb_nested = [[g['gameCode'][-6:][:3], g['gameCode'][-6:][3:]] for g in today_games]
teams_abb = [item for sublist in teams_abb_nested for item in sublist]

In [45]:
detailed_status_to_simple_status = {
    'Out': 'Out',
    'Out For Season': 'Out',
    'Game Time Decision': 'Game Time Decision'
}

In [48]:
injuries_df.loc[:, 'status'] = injuries_df['status'].apply(lambda s: detailed_status_to_simple_status[s])

In [51]:
injuries_df = injuries_df.loc[injuries_df['team'].isin(teams_abb), ['player', 'team', 'status']].sort_values('team')

In [71]:
grouped_injuries_df = injuries_df.groupby(['team', 'status']).agg({'player': ', '.join})

In [83]:
for s in grouped_injuries_df.index.get_level_values('status').unique():
    print(s)

Out
Game Time Decision


In [89]:
grouped_injuries_df.reset_index('status')

,status,player
team,,
ATL,Out,Bogdan Bogdanovic
BKN,Out,T.J. Warren
CHA,Game Time Decision,"Terry Rozier, Cody Martin"
CHA,Out,LaMelo Ball
CHI,Out,"Lonzo Ball, Ayo Dosunmu, Andre Drummond, Justi..."
DAL,Out,"Davis Bertans, Frank Ntilikina, JaVale McGee"
GSW,Game Time Decision,Jordan Poole
GSW,Out,"Andre Iguodala, Donte DiVincenzo"
IND,Out,"Daniel Theis, Myles Turner, Aaron Nesmith, T.J..."
